In [16]:
#Step 1
import pandas as pd
import xlrd
df = pd.read_excel('25045-store-overview.xlsx')

import urllib.request

for index, row in df.iterrows():
    urllib.request.urlretrieve(row['Url'], "output/" + row['Location'] + ".xlsx")

In [ ]:
#Step 2
#Mordor file cleaned up using excel
#=INDEX($I$1:$R$1;1;MATCH("OK";I2:R2;0))

In [157]:
#Step 3
import glob
#glob.glob returns a list of all files in this particular folder
print(glob.glob('output/*'))

DFs = []

for index in glob.glob('output/*'):
    city = str(index).split("/")
    city = city[1].split(".")
    city = city[0]
    print(city)
    DFs.append(pd.read_excel('output/'+city+'.xlsx'))

['output/Norrtälje.xlsx', 'output/Luleå.xlsx', 'output/Linköping.xlsx', 'output/Nässjö.xlsx', 'output/Motala.xlsx', 'output/Kristianstad.xlsx', 'output/Malmö.xlsx', 'output/Ludvika.xlsx', 'output/Falköping.xlsx', 'output/Stockholm.xlsx', 'output/Örebro.xlsx', 'output/Jordbro.xlsx', 'output/Höganäs.xlsx', 'output/Enköping.xlsx', 'output/Mordor.xlsx', 'output/Uppsala.xlsx', 'output/Nordöstra Göteborg.xlsx', 'output/Gislaved.xlsx', 'output/Borås.xlsx', 'output/Jönköping.xlsx', 'output/Gävle.xlsx', 'output/Göteborg.xlsx', 'output/Karlstad.xlsx', 'output/Karlskoga.xlsx', 'output/Växjö.xlsx', 'output/Lund.xlsx', 'output/Tumba.xlsx', 'output/Oskarshamn.xlsx', 'output/Borlänge.xlsx', 'output/Kungälv.xlsx', 'output/Södertälje.xlsx', 'output/Kalmar.xlsx', 'output/Tranås.xlsx', 'output/Halmstad.xlsx', 'output/Norrköping.xlsx']
Norrtälje
Luleå
Linköping
Nässjö
Motala
Kristianstad
Malmö
Ludvika
Falköping
Stockholm
Örebro
Jordbro
Höganäs
Enköping
Mordor
Uppsala
Nordöstra Göteborg
Gislaved
Borås
Jönk

In [77]:
#Step 4
from sqlalchemy import create_engine
host = "mysql-1.cda.hhs.se"
username = "25045"
password = "toystorey" 
schema = "ToyStoreySales"

connection_string = "mysql+pymysql://{}:{}@{}/{}".format(username, password, host, schema)

connection = create_engine(connection_string)

df_receipts = pd.read_sql("select * from Receipt", con=connection)
df_receiptproduct = pd.read_sql("select * from ReceiptProduct", con=connection)
df_products = pd.read_sql("select * from Product", con=connection)

df_rrp = df_receipts.merge(df_receiptproduct, on = "receipt_id", how = "left")

In [212]:
#Step 5
df_rrp["location"] = "online"
df_rrp["sales_rep"] = "NA"

df_rrp = df_rrp[["receipt_product_id", "product_id", "receipt_id", "is_return", "quantity", "location", "date", "payment_method", "sales_rep"]]
df_rrp

df_online = df_rrp
#DFs.append(df_online)
df_complete = pd.concat(DFs)
df_complete

,receipt_product_id,product_id,receipt_id,is_return,quantity,location,date,payment_method,sales_rep
0,da188061-fecf-4477-9066-6df72df52785,PR3341,17,0,1.0,Norrtälje,2014-04-22 16:34:14,Visa,vilgot sundberg
1,f0d947bb-57ea-48f6-85d3-c292fe38c623,PR7066,17,0,2.0,Norrtälje,2014-04-22 16:34:14,Visa,vilgot sundberg
2,2d22405e-bf02-42af-8bb6-4eed54985250,PR1010,17,0,1.0,Norrtälje,2014-04-22 16:34:14,Visa,vilgot sundberg
3,c19a6558-f0cb-4981-85cc-0e24f03337e0,PR1467,61,0,2.0,Norrtälje,2014-04-26 01:03:07,Visa,julie engström
4,6d50d83c-93de-4ad9-8ddb-d71936677475,PR2550,61,0,2.0,Norrtälje,2014-04-26 01:03:07,Visa,julie engström
...,...,...,...,...,...,...,...,...,...
79987,772d2fe5-7bcd-4b20-bba9-c75f21892f12,PR2644,I-9998,0,4.0,online,2020-12-28 15:51:31,Master Card,NA
79988,838e27a4-93f6-4ac2-a021-24120f1ece32,PR4208,I-9998,0,1.0,online,2020-12-28 15:51:31,Master Card,NA
79989,9601cf39-206a-44c2-9f78-45c6bff3360c,PR5,I-9998,0,2.0,online,2020-12-28 15:51:31,Master Card,NA
79990,f740b8ba-f5d0-4a33-ad0b-46e44ac588a5,PR1552,I-9998,0,2.0,online,2020-12-28 15:51:31,Master Card,NA


In [213]:
#Step 6
df_fraudinvestigation = df_complete.groupby(by = 'sales_rep').sum()
df_fraudinvestigation['return_ratio'] = df_fraudinvestigation['is_return'] / df_fraudinvestigation['quantity']
df_potentialfraud = df_fraudinvestigation[df_fraudinvestigation['return_ratio'] > 0.02]
df_potentialfraud.sort_values('return_ratio', ascending = False)
#Beelzebub has a return rate of more than 10% and is therefore a suspect of fraud. All other sales reps have return rates of ~3% or lower

#df_complete = df_complete.merge(df_products, on = 'product_id', how = 'left')
#df_complete
df_complete['revenue'] = df_complete['price'] * df_complete['quantity']
df_Beelzebub = df_complete[df_complete['sales_rep'] == "Beelzebub"]
df_Beelzebub = df_Beelzebub[df_Beelzebub['is_return'] == True]
df_Beelzebub.groupby('sales_rep').sum()
#The sales that Beelzebub has returned sums to 25045 dollars (or other currency unit)

,is_return,quantity,price,cost,revenue
sales_rep,,,,,
Beelzebub,853,8477.0,12524.67,8806.13,25045.0


In [223]:
#Step 7
df_export = df_complete.groupby(['location', 'date', 'receipt_id']).sum()
df_export

df_export.to_excel("output/aggergated-finacial-data.xlsx")
